# So you want to convert Tensorflow to MxNet, eh? 

Lets try parsing the API's and generate a mapping between the two. This might take a while

In [14]:
import mxnet as mx
import tensorflow as tf
import pkgutil, types

def doc_dump(base, docs={}):
    try:
        for a in dir(eval(base)):
            if isinstance(eval(base).__dict__.get(a), types.FunctionType):
                docs[base + '.' + a] = eval(base).__dict__.get(a).__doc__
        for importer, modname, ispkg in pkgutil.walk_packages(path=eval(base).__path__,
                                                              prefix=base + '.',
                                                              onerror=lambda e: None):
            docs = doc_dump(modname, docs)
    except Exception as e:
        pass
    return docs
        
tf_docs = doc_dump('tf', {})
mx_docs = doc_dump('mx', {})